In [39]:
from astropy import units as u
from sunpy.net import Fido
from sunpy.net import attrs as a
import datetime 
import os
import pandas as pd
import sunpy
import sunpy.map
from aiapy.calibrate import normalize_exposure, register, update_pointing

In [2]:
eis_list_file = "../../src/EIS/eis_obs_list.xlsx"

eis_list_df = pd.read_excel(eis_list_file, sheet_name="Sheet1")
eis_list_df

,obs_start,obs_end,texp,stud_acr,repeat,fovx,fovy,cdelt1,des
0,2022-10-19 18:26:00,2022-10-19 22:37:00,10,HH_Flare+AR_180x152H,48,180,152,6,"fast, sparse scan, minimal lines"
1,2022-10-20 03:44:00,2022-10-20 04:47:00,60,DHB_007_v2,1,248,512,4,"slow scan, long slit, large FOV, more lines"
2,2022-10-20 07:05:00,2022-10-20 11:05:00,10,HH_Flare+AR_180x152H,40,180,152,6,"fast, sparse scan, minimal lines"
3,2022-10-20 18:54:00,2022-10-20 20:03:00,10,HH_Flare+AR_180x152H,13,180,152,6,"fast, sparse scan, minimal lines"
4,2022-10-20 23:43:00,2022-10-21 02:12:00,60,DHB_007_v2,2,248,512,4,"slow scan, long slit, large FOV, more lines"
5,2022-10-21 03:15:00,2022-10-21 05:12:00,10,HH_Flare+AR_180x152H,22,180,152,6,"fast, sparse scan, minimal lines"
6,2022-10-21 06:12:00,2022-10-21 17:14:00,10,HH_Flare+AR_180x152H,124,180,152,6,"fast, sparse scan, minimal lines"
7,2022-10-21 18:14:00,2022-10-21 22:41:00,10,HH_Flare+AR_180x152H,50,180,152,6,"fast, sparse scan, minimal lines"
8,2022-10-22 06:30:00,2022-10-22 07:35:00,60,DHB_007_v2,1,248,512,4,"slow scan, long slit, large FOV, more lines"
9,2022-10-22 08:08:00,2022-10-22 10:43:00,10,HH_Flare+AR_180x152H,25,180,152,6,"fast, sparse scan, minimal lines"


In [41]:
aia_download_dir = "../../src/AIA/193/lvl1/"
aia_save_dir = "../../src/AIA/193/lvl15/"

In [44]:
for ii, row_ in eis_list_df[:].iterrows():
    obs_average = row_["obs_start"] + (row_["obs_end"] - row_["obs_start"]) / 2
    

    time_range = a.Time(obs_average, obs_average + datetime.timedelta(seconds=15))

    aia_search = Fido.search(time_range,
                         a.Instrument.aia,
                         a.Wavelength(193*u.angstrom),a.Sample(1*u.minute)) 
    
    downloaded_files = Fido.fetch(aia_search, path=aia_download_dir)

    for downloaded_file in downloaded_files:
        aia_map = sunpy.map.Map(downloaded_file)
        map_updated_pointing = update_pointing(aia_map)
        map_registered = register(map_updated_pointing)
        map_normalized = normalize_exposure(map_registered)

        map_normalized.save(aia_save_dir + os.path.basename(downloaded_file).replace("lev1", "lev15"),overwrite=True)


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia_lev1_193a_2022_10_20t04_15_40_84z_image_lev1.fits:   0%|          | 0.00/11.9M [00:00<?, ?B/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia_lev1_193a_2022_10_20t09_05_04_84z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

...

aia_lev1_193a_2022_10_27t13_21_04_83z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia_lev1_193a_2022_10_28t12_00_40_83z_image_lev1.fits:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia_lev1_193a_2022_10_28t13_31_04_84z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

In [38]:
downloaded_file

['/home/yjzhu/sunpy/data/aia_lev1_193a_2022_10_19t20_31_40_84z_image_lev1.fits']